# Weather Type

The purpose of this notebook is to plot the circulation patterns associated with the various weather types and the time series of weather types

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import calendar
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import cartopy.crs as ccrs
import cartopy.feature
import paraguayfloodspy.visualize as viz # custom plotting library
import matplotlib.patches as patches
%matplotlib inline

Load in configuration parameters from file

In [ ]:
%run ../config/WeatherTypes.mk
print(WTEAST, WTWEST, WTSOUTH, WTNORTH)

Load in plotting options from file

In [ ]:
savefigs = True
%run ../config/PlotParameters.py
print(extent)
print(cmap)

Read in raw data

In [ ]:
psi850 = xr.open_dataset("../data/derived/psi_850.nc")
prcp = xr.open_dataset('../data/derived/precip.nc')
wtypes = pd.read_csv("../data/derived/WeatherTypes.csv", index_col='time', parse_dates=True)

Get the X and Y coordinates for plotting

In [ ]:
X, Y = np.meshgrid(psi850.lon, psi850.lat)
Xp, Yp = np.meshgrid(prcp.lon, prcp.lat)

Print out the climatological probability of each weather type

In [ ]:
wt_counts = wtypes.groupby('wtype').size()
wt_counts2 = wtypes['2015-11-01':'2016-02-29'].groupby('wtype').size()
wt_prop = pd.DataFrame({'climatology': wt_counts / np.sum(wt_counts), 
              'NDJF1516': wt_counts2 / np.sum(wt_counts2)})

In [ ]:
plt.figure(figsize=(8.5, 4.5))
plt.plot(wt_prop['climatology'], label = "Climatology")
plt.plot(wt_prop['NDJF1516'], label = "2015-16")
plt.xlabel("Weather Type")
plt.ylabel("Proportion of Total Days")
plt.legend()
plt.grid()
if savefigs:
    plt.savefig("../figs/WT_Occurrence_Proportion.pdf")

In [ ]:
wt_prop.to_latex("../writeup/WeatherTypeProp.tex")
wt_prop

Repeat for NDJF 2015-16

## Composites

In [ ]:
wt_all = np.unique(wtypes['wtype'])
ncols = len(wt_all)

Give us a box to indicate the weather typing region

In [ ]:
def wt_region():
    rec = patches.Rectangle(
        (WTWEST-360, WTSOUTH),
        WTEAST - WTWEST,
        WTNORTH - WTSOUTH,
        color = 'black', label = 'Weather Typing Region',
        fill = None, linewidth=2,
        transform = ccrs.PlateCarree()
    )
    return(rec)

Create the plot

In [ ]:
fig, axes = viz.SetupAxes(ncol = ncols, nax = 2 * ncols, proj = ccrs.Orthographic(-60, -10), figsize = [18, 5])
for i,wt in enumerate(wt_all):
    def selector(ds):
        times = wtypes.loc[wtypes['wtype'] == wt].index
        ds = ds.sel(time = np.in1d(ds.time, times))
        ds = ds.mean(dim = 'time')
        return(ds)

    # Row 1: 850 hPa wind
    ax = axes[0, i]
    ax.set_title('WT {}: {:.1%} of days'.format(wt, wt_prop['climatology'].values[i]))
    C0 = selector(psi850['anomaly']).plot.contourf(
        transform = ccrs.PlateCarree(), ax=ax,
        cmap = cmap['psi_a'], extend="both", 
        levels=np.linspace(-4.5e6, 4.5e6, 10),
        add_colorbar=False, add_labels=False
    )
    ax.add_patch(wt_region())
    #
    ax = axes[1, i]
    sub = selector(prcp['anomaly']).values
    sub = np.ma.masked_invalid(sub)
    C1 = selector(prcp['anomaly']).plot.contourf(
        transform = ccrs.PlateCarree(), ax=ax,
        cmap = cmap['rain_a'], extend="both", 
        levels=np.linspace(-6, 6, 13),
        add_colorbar=False, add_labels=False
    )
# Colorbars
plt.tight_layout()
fig.subplots_adjust(right=0.94)
cax0 = fig.add_axes([0.97, 0.55, 0.0075, 0.35])
cax2 = fig.add_axes([0.97, 0.05, 0.0075, 0.4])
cbar0 = fig.colorbar(C0, cax = cax0)
cbar0.formatter.set_powerlimits((7, 7))
cbar0.update_ticks() 
cbar0.set_label(r'$\psi_{850}$ Anomaly [$m^2$/s]', rotation=270)
cbar0.ax.get_yaxis().labelpad = 20
cbar1 = fig.colorbar(C1, cax=cax2)
cbar1.set_label('Precip. Anomaly [mm/d]', rotation=270)
cbar1.ax.get_yaxis().labelpad = 20
# Format the axes 
viz.FormatAxes(axes[0,:], extent = extent['SH'])
viz.FormatAxes(axes[1,:], extent = extent['SAm'])
# Save
if savefigs:
    fig.savefig("../figs/WTComposite.pdf", bbox_inches='tight')

## Time Series

In [ ]:
prcp_rpy = xr.open_dataset("../data/derived/rainfall_rpy.nc")['raw'].to_pandas()
prcp_rpy = pd.DataFrame({'raw': prcp_rpy})

In [ ]:
wt_prcp = prcp_rpy.join(wtypes)
wt_prcp = wt_prcp['2015-11-01':'2016-02-29']

In [ ]:
time = wt_prcp.index
rain = wt_prcp.raw.values
wt = np.int_(wt_prcp.wtype.values)

In [ ]:
colors = plt.get_cmap('Accent', 6).colors

In [ ]:
plt.style.use('ggplot')
fig,ax = plt.subplots(nrows=1, ncols=1, figsize=(14,4))
wt_prcp.raw.plot(ax=ax)
ax.axhline(np.nanpercentile(rain, 50), label="p50", color='blue', linestyle='--', linewidth=0.75)
ax.axhline(np.nanpercentile(rain, 90), label="p90", color='blue', linestyle='--', linewidth=0.75)
ax.axhline(np.nanpercentile(rain, 99), label="p99", color='blue', linestyle='--', linewidth=0.75)
ax.set_ylabel('Area-Averaged Rainfall [mm/d]')
ax.grid(True)
for i,t in enumerate(time):
    ax.text(t, rain[i], '{:d}'.format(wt[i]), color=colors[wt[i]-1], size=12, weight='bold')
if savefigs:
    fig.savefig("../figs/RainfallWeatherType.pdf", bbox_inches='tight')